<a href="https://colab.research.google.com/github/krakowiakpawel9/neural-network-course/blob/master/03_keras/01_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Model sekwencjny (Sequential Model):
1. [Wprowadzenie - pierwszy model](#a0)
2. [Funkcje aktywacji](#a1)
3. [Kompilacja modelu](#a2)
4. [Trenowanie modelu](#a3)
5. [Przykład - klasyfikacja binarna](#a4)
6. [Przykład - klasyfikacja wieloklasowa](#a5)
7. [Przykład - regresja](#a6)



Załadowanie biblioteki tensorflow


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import pandas as pd
import plotly.express as px
tf.__version__

TensorFlow 2.x selected.


'2.0.0'

### <a name='a0'></a>Wprowadzenie - pierwszy model
Model sekwencyjny to nic innego jak liniowy stos warstw.

In [0]:
# utworzenie instancji klasy Sequential
from tensorflow.keras.models import Sequential

model = Sequential()
print(model)

Podstawowym elementem składowym modelu są warstwy. Aby dodać najbardziej standardową warstwę - warstwę gęsto połączoną należy użyć warstwy **Dense**. Aby dodać warstwę do modelu należy użyć metody *.add()*

In [0]:
from tensorflow.keras.layers import Dense

model.add(Dense(units=4, input_shape=(10,), ))

Wyświetlenie podsumowania modelu: metoda *.summary()*

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 44        
Total params: 44
Trainable params: 44
Non-trainable params: 0
_________________________________________________________________


Dodanie kolejnej warstwy

In [0]:
model.add(Dense(units=2))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 44        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
Total params: 54
Trainable params: 54
Non-trainable params: 0
_________________________________________________________________


### <a name='a1'></a>Funkcje aktywacji
Istotnym elemenem sieci neuronowych jest dobór odpowiednich funkcji aktywacji. Funkcje aktywacji jak sama nazwa wskazuje są odpowiedzialne za aktywowanie odpowiednich neuronów podczas procesu uczenia.

Jeżeli nie określimy podczas dodawania warstwy funkcji aktywacji, domyślnie stosowana jest liniowa funkcja aktywacji, tzn. $a(x)=x$

Warstwa z liniową funkcją aktywacji może uczyć się tylko liniowych przekształceń danych wejściowych. Dlatego stosuje się różne funkcje aktywacji aby rozwiazywać problemy nieliniowe.

[Keras: Funkcje aktywacji](https://keras.io/activations/)

In [0]:
from tensorflow.keras.activations import linear

random_data = sorted(np.random.randn(200))
data = pd.DataFrame({'data': random_data, 'linear': linear(random_data)})
data.head()

,data,linear
0,-2.554395,-2.554395
1,-2.414174,-2.414174
2,-2.318478,-2.318478
3,-2.290685,-2.290685
4,-2.240938,-2.240938


In [0]:
px.line(data, x='data', y='linear', width=800, range_y=[-2, 2])

In [0]:
from tensorflow.keras.activations import sigmoid

data = pd.DataFrame({'data': random_data, 'sigmoid': sigmoid(random_data)})
data.head()

,data,sigmoid
0,-2.554395,0.072132
1,-2.414174,0.082098
2,-2.318478,0.089604
3,-2.290685,0.091897
4,-2.240938,0.096134


In [0]:
px.line(data, x='data', y='sigmoid', width=800, range_y=[-0.5, 1.5])

In [0]:
from tensorflow.keras.activations import relu

data = pd.DataFrame({'data': random_data, 'relu': relu(random_data)})
data.head()

,data,relu
0,-2.554395,0.0
1,-2.414174,0.0
2,-2.318478,0.0
3,-2.290685,0.0
4,-2.240938,0.0


In [0]:
px.line(data, x='data', y='relu', width=800, range_y=[-0.5, 1.5])

In [0]:
from tensorflow.keras.activations import tanh

data = pd.DataFrame({'data': random_data, 'tanh': tanh(random_data)})
data.head()

,data,tanh
0,-2.554395,-0.987986
1,-2.414174,-0.984128
2,-2.318478,-0.980812
3,-2.290685,-0.979726
4,-2.240938,-0.977629


In [0]:
px.line(data, x='data', y='tanh', width=800, range_y=[-1.5, 1.5])

In [0]:
model = Sequential()
model.add(Dense(units=8, activation='relu', input_shape=(10,)))
model.add(Dense(units=4, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 36        
Total params: 124
Trainable params: 124
Non-trainable params: 0
_________________________________________________________________


### <a name='a2'></a>Kompilacja modelu
Przed rozpoczęciem trenowania sieci należy odpowiednio skonfigurować proces uczenia. W tym kroku określamy:
* rodzaj optymalizatora ([Keras - Optymalizatory](https://keras.io/optimizers/)) 
* funkcję straty ([Keras - Funkcje Straty](https://keras.io/losses/))
* metryki, które będziemy obserwować podczas trenowania sieci ([Keras - Metryki](https://keras.io/metrics/))

In [0]:
# klasyfikacja binarna
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# klasyfikacja wieloklasowa
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# regresja
model.compile(optimizer='rmsprop',
              loss='mse')

### <a name='a3'></a>Trenowanie modelu
Za dane wejściowe do modelu należy przekazać Numpy arrays:
* **epochs** - krotność przejścia danych przez sieć w procesie uczenia
* **batch_size** - rozmiar wsadu po którym następuje aktualizacja wag
* **validation_split** - część danych treningowych, które zostaną wykorzystane jako zbiór walidacyjny
* **validation_data** - (x_val, y_val) - dane wykorzystane do walidacji modelu

In [0]:
# model.fit(data, labels, epoch=10, batch_size=32)
# model.fit(data, labels, epoch=10, batch_size=32, validation_split=0.2)
# model.fit(data, labels, epoch=10, batch_size=32, validation_data=(x_val, y_val))

### <a name='a4'></a> Przykład - klasyfikacja binarna

In [0]:
data = np.random.randn(1000, 150)
labels = np.random.randint(2, size=(1000, 1))

print(data.shape)
print(labels.shape)

(1000, 150)
(1000, 1)


In [0]:
data[:3]

array([[ 8.85759772e-01, -3.65118301e-01,  7.43365868e-02,
        -2.17827899e-01,  9.39047258e-02,  1.87847027e+00,
         1.06544272e+00,  5.78634097e-01,  5.69670980e-01,
         9.35686357e-01,  1.56070690e+00, -1.43237599e-01,
        -1.85858231e-02, -4.84553287e-01, -4.96946874e-01,
         1.56536516e+00,  5.13897854e-01, -1.10256143e+00,
        -4.99904511e-01, -1.18690698e+00, -2.17797539e-01,
         1.74776180e+00,  1.15659587e+00, -1.97464313e+00,
         6.48162263e-02, -2.03554715e-01,  9.67050475e-01,
        -1.07694145e+00,  4.47607211e-01,  5.44315197e-01,
         4.36198222e-01, -2.07815839e-03,  7.81715413e-01,
        -2.35708453e-01,  1.68208641e+00, -1.42411854e+00,
        -7.39091441e-01, -1.00232827e+00, -5.81600648e-01,
        -8.35185573e-01, -4.09283199e-01,  2.77245469e-01,
         2.47926828e-01,  1.13199948e-01,  4.47539261e-01,
         4.97107472e-01,  1.76112491e+00, -2.33217577e-01,
        -6.12208512e-01, -1.49912202e-01, -5.51033670e-0

In [0]:
labels[:10]

array([[1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1]])

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20)

Train on 1000 samples
Epoch 1/20
1000/1000 [==============================] - 1s 976us/sample - loss: 0.8453 - accuracy: 0.4620
Epoch 2/20
1000/1000 [==============================] - 0s 57us/sample - loss: 0.7361 - accuracy: 0.5340
Epoch 3/20
1000/1000 [==============================] - 0s 56us/sample - loss: 0.6814 - accuracy: 0.5800
Epoch 4/20
1000/1000 [==============================] - 0s 70us/sample - loss: 0.6403 - accuracy: 0.6320
Epoch 5/20
1000/1000 [==============================] - 0s 63us/sample - loss: 0.6055 - accuracy: 0.6730
Epoch 6/20
1000/1000 [==============================] - 0s 72us/sample - loss: 0.5733 - accuracy: 0.7160
Epoch 7/20
1000/1000 [==============================] - 0s 70us/sample - loss: 0.5455 - accuracy: 0.7530
Epoch 8/20
1000/1000 [==============================] - 0s 66us/sample - loss: 0.5180 - accuracy: 0.7890
Epoch 9/20
1000/1000 [==============================] - 0s 64us/sample - loss: 0.4935 - accuracy: 0.8160
Epoch 10/20
1000/1000 [=========

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=30)

Train on 1000 samples
Epoch 1/20
1000/1000 [==============================] - 1s 589us/sample - loss: 0.7704 - accuracy: 0.4910
Epoch 2/20
1000/1000 [==============================] - 0s 69us/sample - loss: 0.6936 - accuracy: 0.5610
Epoch 3/20
1000/1000 [==============================] - 0s 72us/sample - loss: 0.6511 - accuracy: 0.6140
Epoch 4/20
1000/1000 [==============================] - 0s 73us/sample - loss: 0.6128 - accuracy: 0.6600
Epoch 5/20
1000/1000 [==============================] - 0s 73us/sample - loss: 0.5823 - accuracy: 0.6930
Epoch 6/20
1000/1000 [==============================] - 0s 76us/sample - loss: 0.5537 - accuracy: 0.7280
Epoch 7/20
1000/1000 [==============================] - 0s 72us/sample - loss: 0.5262 - accuracy: 0.7720
Epoch 8/20
1000/1000 [==============================] - 0s 75us/sample - loss: 0.5004 - accuracy: 0.7930
Epoch 9/20
1000/1000 [==============================] - 0s 75us/sample - loss: 0.4755 - accuracy: 0.8140
Epoch 10/20
1000/1000 [=========

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/20
800/800 [==============================] - 1s 946us/sample - loss: 0.8127 - accuracy: 0.4913 - val_loss: 0.7460 - val_accuracy: 0.4850
Epoch 2/20
800/800 [==============================] - 0s 84us/sample - loss: 0.7164 - accuracy: 0.5462 - val_loss: 0.7465 - val_accuracy: 0.4700
Epoch 3/20
800/800 [==============================] - 0s 85us/sample - loss: 0.6626 - accuracy: 0.5875 - val_loss: 0.7526 - val_accuracy: 0.4650
Epoch 4/20
800/800 [==============================] - 0s 83us/sample - loss: 0.6202 - accuracy: 0.6438 - val_loss: 0.7631 - val_accuracy: 0.4700
Epoch 5/20
800/800 [==============================] - 0s 78us/sample - loss: 0.5833 - accuracy: 0.6900 - val_loss: 0.7766 - val_accuracy: 0.4750
Epoch 6/20
800/800 [==============================] - 0s 97us/sample - loss: 0.5517 - accuracy: 0.7237 - val_loss: 0.7909 - val_accuracy: 0.4700
Epoch 7/20
800/800 [==============================] - 0s 88us/sample - loss: 0.5224

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

In [0]:
metrics = history.history
metrics.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

Train on 800 samples, validate on 200 samples
Epoch 1/20
800/800 [==============================] - 1s 1ms/sample - loss: 0.7561 - accuracy: 0.5200 - val_loss: 0.7644 - val_accuracy: 0.4700
Epoch 2/20
800/800 [==============================] - 0s 96us/sample - loss: 0.6723 - accuracy: 0.6025 - val_loss: 0.7633 - val_accuracy: 0.4700
Epoch 3/20
800/800 [==============================] - 0s 90us/sample - loss: 0.6237 - accuracy: 0.6488 - val_loss: 0.7708 - val_accuracy: 0.4650
Epoch 4/20
800/800 [==============================] - 0s 84us/sample - loss: 0.5838 - accuracy: 0.7075 - val_loss: 0.7823 - val_accuracy: 0.4700
Epoch 5/20
800/800 [==============================] - 0s 84us/sample - loss: 0.5489 - accuracy: 0.7312 - val_loss: 0.7984 - val_accuracy: 0.4600
Epoch 6/20
800/800 [==============================] - 0s 86us/sample - loss: 0.5174 - accuracy: 0.7725 - val_loss: 0.8164 - val_accuracy: 0.4400
Epoch 7/20
800/800 [==============================] - 0s 81us/sample - loss: 0.4878 -

In [0]:
test_data = np.random.randn(5, 150)
test_labels = np.random.randint(2, size=(5, 1))

In [0]:
model.predict(test_data)

array([[0.52326953],
       [0.23457848],
       [0.62082064],
       [0.46661398],
       [0.29559034]], dtype=float32)

In [0]:
model.predict_proba(test_data)

array([[0.52326953],
       [0.23457848],
       [0.62082064],
       [0.46661398],
       [0.29559034]], dtype=float32)

In [0]:
model.predict_classes(test_data)

array([[1],
       [0],
       [1],
       [0],
       [0]], dtype=int32)

### <a name='a5'></a> Przykład - klasyfikacja wieloklasowa

In [0]:
data = np.random.random((1000, 150))
labels = np.random.randint(10, size=(1000, 1))

In [0]:
print(data.shape)
print(labels.shape)

(1000, 150)
(1000, 1)


In [0]:
data[:3]

array([[0.99760735, 0.06208589, 0.03693355, 0.54656407, 0.3287549 ,
        0.05714941, 0.56801429, 0.01178218, 0.79517162, 0.29410566,
        0.42673894, 0.63171455, 0.43536109, 0.65690796, 0.39826716,
        0.52386633, 0.24427354, 0.0438452 , 0.47607657, 0.50803553,
        0.91960684, 0.32871824, 0.79534731, 0.48301448, 0.49657061,
        0.43454219, 0.13096995, 0.31551124, 0.02915482, 0.38102174,
        0.22947491, 0.83675642, 0.09845884, 0.08829755, 0.02137274,
        0.3869031 , 0.05612117, 0.78761232, 0.80193592, 0.36686429,
        0.83641619, 0.48283622, 0.07948118, 0.64096124, 0.30504208,
        0.75694303, 0.4909508 , 0.13756621, 0.79060454, 0.81297295,
        0.38297125, 0.66721962, 0.02027357, 0.73192813, 0.7095527 ,
        0.40803589, 0.46712719, 0.07904283, 0.58440089, 0.15886942,
        0.9578673 , 0.84052242, 0.10961125, 0.23542429, 0.6803045 ,
        0.76927152, 0.73931983, 0.50734059, 0.49598425, 0.40783233,
        0.59146463, 0.82890758, 0.43088351, 0.18

In [0]:
labels[:10]

array([[0],
       [3],
       [6],
       [6],
       [5],
       [2],
       [2],
       [3],
       [9],
       [7]])

In [0]:
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels, num_classes=10)
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [0]:
labels[1]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=30, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/30
800/800 [==============================] - 1s 754us/sample - loss: 2.3707 - accuracy: 0.0850 - val_loss: 2.3458 - val_accuracy: 0.1050
Epoch 2/30
800/800 [==============================] - 0s 87us/sample - loss: 2.3224 - accuracy: 0.1037 - val_loss: 2.3404 - val_accuracy: 0.1050
Epoch 3/30
800/800 [==============================] - 0s 91us/sample - loss: 2.3004 - accuracy: 0.1125 - val_loss: 2.3253 - val_accuracy: 0.1150
Epoch 4/30
800/800 [==============================] - 0s 87us/sample - loss: 2.2859 - accuracy: 0.1587 - val_loss: 2.3401 - val_accuracy: 0.1200
Epoch 5/30
800/800 [==============================] - 0s 84us/sample - loss: 2.2748 - accuracy: 0.1312 - val_loss: 2.3398 - val_accuracy: 0.1450
Epoch 6/30
800/800 [==============================] - 0s 91us/sample - loss: 2.2558 - accuracy: 0.1637 - val_loss: 2.3336 - val_accuracy: 0.1000
Epoch 7/30
800/800 [==============================] - 0s 81us/sample - loss: 2.2356

In [0]:
test_data = np.random.random((10, 150))

model.predict(test_data)

array([[0.0287402 , 0.14490484, 0.07091898, 0.16534875, 0.08404998,
        0.08279553, 0.26000288, 0.0633504 , 0.0256419 , 0.07424662],
       [0.07591555, 0.1340096 , 0.11344802, 0.0851059 , 0.0750747 ,
        0.20681165, 0.15166637, 0.04493193, 0.04997318, 0.063063  ],
       [0.07848288, 0.10386084, 0.02226465, 0.13035418, 0.06963193,
        0.05490661, 0.0446665 , 0.13471058, 0.11770166, 0.24342018],
       [0.16647425, 0.13040043, 0.03104859, 0.13561353, 0.03981173,
        0.07594257, 0.06423546, 0.07862286, 0.06769135, 0.21015923],
       [0.28612733, 0.08047947, 0.04853185, 0.07047633, 0.11869612,
        0.13276224, 0.09324729, 0.03398637, 0.0455109 , 0.09018212],
       [0.06013485, 0.1452705 , 0.0319569 , 0.23275879, 0.09412711,
        0.05648237, 0.04654945, 0.1297133 , 0.090859  , 0.11214773],
       [0.11729616, 0.13364924, 0.04253459, 0.18120545, 0.03903491,
        0.09838022, 0.16830875, 0.06120457, 0.08364568, 0.07474031],
       [0.08277321, 0.07184754, 0.0957608

In [0]:
model.predict_classes(test_data)

array([6, 5, 9, 9, 0, 3, 3, 6, 0, 4])

### <a name='a6'></a> Przykład - regresja

In [0]:
data = np.random.random((1000, 150))
labels = 50 * np.random.random(1000)

In [0]:
data[:3]

array([[0.02026823, 0.63841918, 0.74923883, 0.77005058, 0.10351624,
        0.57493033, 0.23548592, 0.66298468, 0.47367352, 0.96200758,
        0.34917653, 0.41757657, 0.86798714, 0.00336975, 0.59901289,
        0.18995215, 0.18951166, 0.74857895, 0.68201817, 0.23749761,
        0.03939157, 0.11201925, 0.66290513, 0.25489255, 0.66354499,
        0.68749734, 0.57616677, 0.12944316, 0.97974155, 0.96251269,
        0.69677507, 0.70021379, 0.37738098, 0.55274783, 0.57381456,
        0.35486382, 0.71492671, 0.61341747, 0.02514807, 0.76471124,
        0.20829556, 0.16374408, 0.31751839, 0.67925924, 0.32686336,
        0.01505189, 0.73231972, 0.19208245, 0.56759105, 0.71583639,
        0.25154403, 0.03596326, 0.27366251, 0.02952513, 0.49045917,
        0.29910135, 0.26394726, 0.23532939, 0.3478145 , 0.52468155,
        0.43489956, 0.04737573, 0.22275554, 0.45758452, 0.69196932,
        0.07509749, 0.01539229, 0.74396923, 0.81293404, 0.36570207,
        0.46964689, 0.98024356, 0.72618946, 0.39

In [0]:
labels[:10]

array([ 9.81534792, 47.02646867, 16.36161199, 32.25346502, 10.26476918,
       18.78263068, 19.47096313,  9.4102452 , 21.62531934, 29.13568997])

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mse')

model.fit(data, labels, epochs=30, batch_size=32, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/30
800/800 [==============================] - 0s 610us/sample - loss: 542.1807 - val_loss: 369.4364
Epoch 2/30
800/800 [==============================] - 0s 81us/sample - loss: 277.1112 - val_loss: 220.0210
Epoch 3/30
800/800 [==============================] - 0s 78us/sample - loss: 209.1723 - val_loss: 203.2772
Epoch 4/30
800/800 [==============================] - 0s 78us/sample - loss: 205.8558 - val_loss: 203.4024
Epoch 5/30
800/800 [==============================] - 0s 89us/sample - loss: 204.7815 - val_loss: 205.6609
Epoch 6/30
800/800 [==============================] - 0s 82us/sample - loss: 204.5070 - val_loss: 203.3121
Epoch 7/30
800/800 [==============================] - 0s 77us/sample - loss: 203.7743 - val_loss: 203.8386
Epoch 8/30
800/800 [==============================] - 0s 81us/sample - loss: 202.9408 - val_loss: 204.2966
Epoch 9/30
800/800 [==============================] - 0s 83us/sample - loss: 202.4409 - val_loss:

In [0]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

model.fit(data, labels, epochs=30, batch_size=32, validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/30
800/800 [==============================] - 1s 1ms/sample - loss: 19.5757 - mse: 553.8293 - val_loss: 16.7935 - val_mse: 417.4047
Epoch 2/30
800/800 [==============================] - 0s 92us/sample - loss: 15.0221 - mse: 331.6117 - val_loss: 13.7133 - val_mse: 261.1456
Epoch 3/30
800/800 [==============================] - 0s 84us/sample - loss: 12.8232 - mse: 230.6383 - val_loss: 12.6731 - val_mse: 208.1648
Epoch 4/30
800/800 [==============================] - 0s 88us/sample - loss: 12.3133 - mse: 206.6382 - val_loss: 12.5968 - val_mse: 204.2679
Epoch 5/30
800/800 [==============================] - 0s 85us/sample - loss: 12.2768 - mse: 206.0411 - val_loss: 12.5878 - val_mse: 204.1514
Epoch 6/30
800/800 [==============================] - 0s 81us/sample - loss: 12.2775 - mse: 206.0161 - val_loss: 12.5908 - val_mse: 204.0778
Epoch 7/30
800/800 [==============================] - 0s 78us/sample - loss: 12.2421 - mse: 205.5852 - val_lo

In [0]:
test_data = np.random.random((10, 150))

model.predict(test_data)

array([[24.83885 ],
       [19.244923],
       [28.465197],
       [26.268496],
       [25.207014],
       [25.766394],
       [28.84799 ],
       [25.306421],
       [25.532907],
       [27.390581]], dtype=float32)